# Amplitude do gradiente direcional da anomalia de campo total

Este código utiliza a biblioteca de modelagem e inversão de dados geofísicos denominada [Fatiando a Terra](http://www.fatiando.org/index.html).

### Coisas para importar

In [1]:
#%matplotlib inline
import numpy as np
from fatiando import mesher, gridder, utils
from fatiando.gravmag import polyprism
from fatiando.vis import mpl

### Pontos onde são calculados os dados

In [2]:
shape = (100, 100) # número de pontos ao longo de x e y
area = [-5000, 5000, -5000, 5000] # x mínimo, x máximo, y mínimo e y máximo da área
xp, yp, zp = gridder.regular(area, shape, z=-100) # grade regular localizada em z = -100 m

xc = 0.5*(area[1] + area[0])
yc = 0.5*(area[3] + area[2])

### Inclinação e a declinação do campo geomagnético local

In [3]:
inc = 56. # graus
dec = -60. # graus
amp = 23500.0 # nT

In [4]:
F = utils.ang2vec(amp, inc, dec) # vetor campo geomagnético local

In [5]:
print F

[  6570.51661578 -11380.46861051  19482.38295504]


### Modelo

In [6]:
mpl.close('all')
axis = mpl.figure().gca()
mpl.axis('scaled')
vertices = mpl.draw_polygon(area, axis, xy2ne=True)

In [7]:
print vertices

[[  3.32112971e+03  -3.00732218e+03]
 [  2.69351464e+03  -2.95502092e+03]
 [  2.01359833e+03  -2.77196653e+03]
 [  1.54288703e+03  -2.53661088e+03]
 [  1.20292887e+03  -1.90899582e+03]
 [  1.01987448e+03  -1.09832636e+03]
 [  9.93723849e+02  -2.61506276e+01]
 [  6.53765690e+02   7.06066946e+02]
 [  1.83054393e+02   1.15062762e+03]
 [ -4.18410042e+02   1.67364017e+03]
 [ -1.01987448e+03   1.98744770e+03]
 [ -1.85669456e+03   2.43200837e+03]
 [ -2.17050209e+03   2.58891213e+03]
 [ -2.61506276e+03   3.08577406e+03]
 [ -2.92887029e+03   3.63493724e+03]
 [ -3.29497908e+03   4.21025105e+03]
 [ -3.89644351e+03   4.47175732e+03]
 [ -4.10564854e+03   4.18410042e+03]
 [ -4.05334728e+03   3.76569038e+03]
 [ -3.50418410e+03   3.19037657e+03]
 [ -3.05962343e+03   2.66736402e+03]
 [ -2.35355649e+03   1.93514644e+03]
 [ -1.49058577e+03   1.54288703e+03]
 [ -7.06066946e+02   1.04602510e+03]
 [ -1.30753138e+02   2.87656904e+02]
 [  9.09494702e-13   0.00000000e+00]
 [  4.18410042e+02  -1.22907950e+03]
 

In [8]:
incs = inc # inclinacao da magnetizacao do corpo (em graus)
decs = 0. # declinacao da magnetizacao do corpo (em graus)
amps = 10. # intensidade da magnetizacao do corpo (em A/m)

In [9]:
model = [mesher.PolygonalPrism(vertices, 10, 1010, {'magnetization': utils.ang2vec(amps, incs, decs)})]

### Anomalia de campo total (em nT)

In [10]:
act = polyprism.tf(xp, yp, zp, model, inc, dec)

### Derivadas espaciais da anomalia de campo total (em nT/m)

In [11]:
delta = 10.

dx_act = (polyprism.tf(xp + delta, yp, zp, model, inc, dec) - \
          polyprism.tf(xp - delta, yp, zp, model, inc, dec))/(2.*delta)

dy_act = (polyprism.tf(xp, yp + delta, zp, model, inc, dec) - \
          polyprism.tf(xp, yp - delta, zp, model, inc, dec))/(2.*delta)

dz_act = (polyprism.tf(xp, yp, zp + delta, model, inc, dec) - \
          polyprism.tf(xp, yp, zp - delta, model, inc, dec))/(2.*delta)

In [16]:
teta = np.deg2rad(135.)

dv_act = np.cos(teta)*dx_act + np.sin(teta)*dy_act

### Amplitude do gradiente total

In [13]:
grad = np.sqrt(dx_act**2 + dy_act**2 + dz_act**2)

### Plot das derivadas ao longo dos eixos x, y e z

In [17]:
mpl.close('all')
mpl.figure(figsize=(14,10))

mpl.subplot(2,2,1)
mpl.title("Derivada direcional da anomalia de campo total (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, dv_act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(2,2,2)
mpl.title("Derivada em x da anomalia de campo total (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, dx_act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(2,2,3)
mpl.title("Derivada em y da anomalia de campo total (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, dy_act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(2,2,4)
mpl.title("Derivada em z da anomalia de campo total (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, dz_act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.tight_layout()

mpl.savefig('prismas_mag_derivadas.png', width=1200)

#mpl.show()

<img src='prismas_mag_derivadas.png', width=1200>

In [15]:
mpl.close('all')
mpl.figure(figsize=(14,5))

mpl.subplot(1,2,1)
mpl.title("Anomalia de campo total (nT)")
mpl.axis('scaled')
mpl.contourf(yp, xp, act, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.subplot(1,2,2)
mpl.title("Amplitude do gradiente (nT/m)")
mpl.axis('scaled')
mpl.contourf(yp, xp, grad, shape, 15, cmap = mpl.get_cmap('RdBu_r'))
mpl.colorbar()
mpl.polygon(model[0], '--k', xy2ne=True)
mpl.xlabel('y (km)')
mpl.ylabel('x (km)')
mpl.m2km()

mpl.savefig('prismas_mag_grad_total.png', width=1200)

<img src='prismas_mag_grad_total.png', width=1200>